In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tqdm import tqdm
import random

In [3]:
import malaya

In [4]:
# model = malaya.dependency.transformer(model = 'xlnet', quantized = True)

In [5]:
penguat_list = [
    'paling',
    'agak',
    'sungguh',
    'amat',
    'terlalu',
    'nian',
    'benar',
    'paling',
    'sangat'
]
end_penguat_list = ['sekali', 'sungguh', 'sangat']

In [6]:
set_penguat_list = set(penguat_list)

In [7]:
with open('filtered-dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]

len(data)

2037249

In [8]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize
sastrawi = malaya.stem.sastrawi()

In [9]:
def check_start_ter(word):
    stemmed = sastrawi.stem(word)
    if word.startswith('ter') and not stemmed.startswith('ter') and stemmed in word:
        return True
    return False

In [10]:
check_start_ter('tertinggi'), check_start_ter('terjalan')

(True, False)

In [11]:
# graph, tagging, indexing = model.predict(' '.join(tokenizer('ibu bapa kita bekerja penat lelah')))
# graph.to_graphvis()

In [12]:
import copy
import re
from malaya.text.regex import _expressions

def reset_t(tokens):
    t = []
    for i in range(len(tokens)):
        t.append([tokens[i], 2])
    return t

# [ter][kata sifat] -> [ter][kata sifat] [end_penguat_list]
def augment_6_0(t, row):
    text, tokens, tokens_lower, penguat = row
    for i in range(len(tokens)):
        if check_start_ter(tokens[i]):
            ends = random.choice(end_penguat_list)
            t[i][0] = f'{tokens[i]} {ends}'
            t[i][1] = 6

In [13]:
results = []
for text in tqdm(['dapat markah yang tertinggi']):
    tokens = tokenizer(text)
    t = reset_t(tokens)
    t_ = copy.deepcopy(t)
    tokens_lower = tokenizer(text.lower())
    set_tokens = set(tokens_lower)
    r_penguat_list = set_tokens & set_penguat_list
    r = (t, tokens, tokens_lower, r_penguat_list)
    augment_6_0(t_, r)
    results.append((t, t_))

100%|██████████| 1/1 [00:00<00:00, 283.02it/s]


In [14]:
results

[([['dapat', 2], ['markah', 2], ['yang', 2], ['tertinggi', 2]],
  [['dapat', 2], ['markah', 2], ['yang', 2], ['tertinggi sekali', 6]])]